In [ ]:
#install.packages("tidyverse")
library(tidyverse) # loads dplyr, ggplot2, stringr etc

#install.packages(c("car","agricolae", "FSA"))
library(car)
library(agricolae)
library(FSA)

In [ ]:
#Jupyter root Oulema-Wheat
pliman <- read.csv("2_leaf_damage.csv", header = TRUE)

In [ ]:
##STATS##–––––––– 

# CHECKING NORMAL DISTRIBUTION

# FOR REGULAR DATA -- Check assumptions: normality and homogeneity of variances

shapiro = as.numeric(shapiro.test(pliman$Damaged)$p.value)
levene = as.numeric(car::leveneTest(Damaged ~ Soil, data=pliman)$`Pr(>F)`[1])
if (shapiro > 0.05 & levene > 0.05) {
  print("     you should perform ANOVA test")
} else {
  print("    you should perform Kruskal-Wallis test")
}
print(paste0("Shapiro test p-value: ", shapiro))
print(paste0("Levene test p-value: ", levene))

kruskal.test(Damaged~Treatment, data = pliman) # significant difference between control and insect

In [ ]:
#Testing differences between conducive and suppressive
wil <- data.frame(type = pliman$Soil_Type, damage = pliman$Damaged)
wil <- wil %>% arrange(type)
wil$id <- c(1:109,1:110)
new.wil <- reshape(wil, idvar = "id", timevar = "type", direction = "wide")
names(new.wil) <- c("id", "conducive", "suppressive")

wilcox.test(new.wil$conducive, new.wil$suppressive)
#There is no statistically significant difference between conducive and suppressive

In [ ]:
pliman$Condition <- paste0(pliman$Soil, "_", pliman$Treatment)

    kw = agricolae::kruskal(pliman$Damaged, 
                        pliman$Condition, alpha = 0.05, p.adj = "fdr")
    kwres = rownames_to_column(kw$groups, var="Name")

plot <- ggplot(pliman, aes(x=Condition, y=Damaged, fill=Soil)) +
    scale_fill_manual(values= c("#887bb9", "#a35da5", "#69b19a", "#8ec372")) +
    geom_boxplot(outlier.alpha = 0.8, outlier.shape = 16) + 
    geom_jitter(alpha=0.9, size=0.4, width = 0.5, shape=16) +
    geom_vline(xintercept=c(2.5,4.5,6.5), 
               linetype = "dashed", color = "grey") +
    annotate(geom = "text", x=kwres$Name, 
             y = 110, label=kwres$groups) +
    theme_classic() + theme(axis.text.x = element_text(angle=45, vjust=1, hjust=1))

    print(plot)